In [59]:
from dagster import EnvVar
from pyiceberg.catalog import load_catalog

name = "default"
warehouse = EnvVar("NESSIE_WAREHOUSE").get_value()
branch = EnvVar("NESSIE_BRANCH").get_value()
uri = EnvVar("NESSIE_URI").get_value()
py_io_impl = "pyiceberg.io.pyarrow.PyArrowFileIO"
s3_endpoint = EnvVar("DESTINATION__FILESYSTEM__CREDENTIALS__AWS_S3_ENDPOINT").get_value()
s3_access_key_id = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_ACCESS_KEY_ID"
).get_value()
s3_secret_access_key = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_SECRET_ACCESS_KEY"
).get_value()
catalog_type = "rest"
catalog = load_catalog(
            **{
                "warehouse": warehouse,
                "uri": f"{uri}/{branch}",
                "py-io-impl": py_io_impl,
                "s3.endpoint": s3_endpoint,
                "s3.access-key-id": s3_access_key_id,
                "s3.secret-access-key": s3_secret_access_key,
                "type": catalog_type,
            },
        )

print(branch)

feat-update-schema


In [60]:
catalog.list_tables("staging")

[('staging', 'cit0day_password_files')]

In [61]:
#table = catalog.drop_table("staging.cit0day_password_files")

# Query the Data

In [62]:
table = catalog.load_table("staging.cit0day_password_files")

In [63]:
con = table.scan(limit=1000).to_duckdb(table_name="cit0day_password_files")
df = pl.DataFrame(con.execute(
        "select * from cit0day_password_files"
    ).fetch_arrow_table())

In [64]:
print(df)

shape: (1_000, 7)
┌────────────────┬───────────────┬───────────────┬─────────────┬────────┬───────────────┬──────────┐
│ email          ┆ username      ┆ domain        ┆ data        ┆ bucket ┆ prefix        ┆ category │
│ ---            ┆ ---           ┆ ---           ┆ ---         ┆ ---    ┆ ---           ┆ ---      │
│ str            ┆ str           ┆ str           ┆ str         ┆ str    ┆ str           ┆ str      │
╞════════════════╪═══════════════╪═══════════════╪═════════════╪════════╪═══════════════╪══════════╡
│ yadihomefurnit ┆ yadihomefurni ┆ 02zillions.co ┆ afobycyc    ┆ raw    ┆ extracted/Cit ┆ Business │
│ ures@02zillion ┆ tures         ┆ m             ┆             ┆        ┆ 0day Prem     ┆          │
│ s.…            ┆               ┆               ┆             ┆        ┆ [_speci…      ┆          │
│ kerrycavill@08 ┆ kerrycavill   ┆ 0815.ru       ┆ WN9AAiln    ┆ raw    ┆ extracted/Cit ┆ Business │
│ 15.ru          ┆               ┆               ┆             ┆        ┆

In [50]:
df["prefix"][0]

'extracted/Cit0day Prem [_special_for_xss.is]/0-deductible-offer.ca {6.992} [HASH+NOHASH] (NoCategory)_special_for_XSS.IS/0-deductible-offer.ca {6.992} decrypted.txt'